In [28]:
liked_books = ['12961964', '18887980', '18902634', '15769088', '28862387']

In [29]:
# This is user-based CF. Basically, we find a list of users who have liked similar books to the current user. Then we can make recommendations based on what other books these users liked.

book_mapping = {}

with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        book_mapping[csv_id] = book_id

In [30]:
overlap_users = set()
# overlap_users contains a list of users who liked the books
with open("goodreads_interactions.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue
        
        try:
            rating = int(rating)
        except ValueError:
            continue
        book_id = book_mapping[csv_id]
        if book_id in liked_books and rating >=4:
            overlap_users.add(user_id)

In [31]:
recs = []
# recs contains books that users who liked similar books have read.
with open("goodreads_interactions.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = book_mapping[csv_id]
            recs.append([user_id, book_id, rating])

In [32]:
import pandas as pd

rec = pd.DataFrame(recs, columns = ['user_id', 'book_id', 'rating'])
rec['book_id'] = rec['book_id'].astype(str)

In [33]:
top_recs = rec['book_id'].value_counts().head(10) #counts up number of times each book occurred. shows top books
top_recs = top_recs.index.values

In [34]:
book_titles = pd.read_json("book_titles.json")
book_titles["book_id"] = book_titles["book_id"].astype(str)

In [35]:
book_titles[book_titles["book_id"].isin(top_recs)]

,book_id,title,ratings,url,cover,modified_title
65983,7260188,"Mockingjay (The Hunger Games, #3)",1743362,https://www.goodreads.com/book/show/7260188-mo...,https://images.gr-assets.com/books/1358275419m...,mockingjay the hunger games 3
255053,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
289541,41865,"Twilight (Twilight, #1)",3941381,https://www.goodreads.com/book/show/41865.Twil...,https://images.gr-assets.com/books/1361039443m...,twilight twilight 1
415023,4671,The Great Gatsby,2758812,https://www.goodreads.com/book/show/4671.The_G...,https://images.gr-assets.com/books/1490528560m...,the great gatsby
550784,5470,1984,2023937,https://www.goodreads.com/book/show/5470.1984,https://images.gr-assets.com/books/1348990566m...,1984
708106,2657,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
746012,6148028,"Catching Fire (The Hunger Games, #2)",1854746,https://www.goodreads.com/book/show/6148028-ca...,https://images.gr-assets.com/books/1358273780m...,catching fire the hunger games 2
786647,3,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
808272,136251,Harry Potter and the Deathly Hallows (Harry Po...,1784684,https://www.goodreads.com/book/show/136251.Har...,https://images.gr-assets.com/books/1474171184m...,harry potter and the deathly hallows harry pot...
1142466,12961964,The Handmaid's Tale,39362,https://www.goodreads.com/book/show/12961964-t...,https://images.gr-assets.com/books/1498139932m...,the handmaids tale


In [47]:
all_recs = rec["book_id"].value_counts()

In [48]:
all_recs = all_recs.to_frame().reset_index()

In [49]:
all_recs.columns = ["book_id", "book_count"]

In [50]:
all_recs = all_recs.merge(book_titles, how="inner", on="book_id")

In [51]:
all_recs

,book_id,book_count,title,ratings,url,cover,modified_title
0,2767052,2687,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
1,12961964,2679,The Handmaid's Tale,39362,https://www.goodreads.com/book/show/12961964-t...,https://images.gr-assets.com/books/1498139932m...,the handmaids tale
2,3,2516,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
3,2657,2401,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
4,4671,2191,The Great Gatsby,2758812,https://www.goodreads.com/book/show/4671.The_G...,https://images.gr-assets.com/books/1490528560m...,the great gatsby
...,...,...,...,...,...,...,...
411998,13576148,1,"Quest, Inc.",25,https://www.goodreads.com/book/show/13576148-q...,https://images.gr-assets.com/books/1333471151m...,quest inc
411999,13389198,1,"The Teenage Chronicles (Creature Kids, #1)",21,https://www.goodreads.com/book/show/13389198-t...,https://images.gr-assets.com/books/1327953483m...,the teenage chronicles creature kids 1
412000,13265776,1,"Demon Gates (Nexus Wars Saga, #1)",97,https://www.goodreads.com/book/show/13265776-d...,https://images.gr-assets.com/books/1327885981m...,demon gates nexus wars saga 1
412001,7220447,1,Memoirs of an Antihero,146,https://www.goodreads.com/book/show/7220447-me...,https://images.gr-assets.com/books/1329665942m...,memoirs of an antihero


In [52]:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"]) #penalizes popularity so that the recs arent always books with the most ratings
all_recs.sort_values("score", ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover,modified_title,score
1,12961964,2679,The Handmaid's Tale,39362,https://www.goodreads.com/book/show/12961964-t...,https://images.gr-assets.com/books/1498139932m...,the handmaids tale,182.334256
65,15769088,1013,Pride and Prejudice,17631,https://www.goodreads.com/book/show/15769088-p...,https://images.gr-assets.com/books/1364431117m...,pride and prejudice,58.202541
10055,36307629,32,"King of Scars (King of Scars, #1)",22,https://www.goodreads.com/book/show/36307629-k...,https://images.gr-assets.com/books/1506962795m...,king of scars king of scars 1,46.545455
4301,28170940,66,"Lethal White (Cormoran Strike, #4)",106,https://www.goodreads.com/book/show/28170940-l...,https://s.gr-assets.com/assets/nophoto/book/11...,lethal white cormoran strike 4,41.094340
38,18902634,1290,The Goldfinch,41942,https://www.goodreads.com/book/show/18902634-t...,https://images.gr-assets.com/books/1488318482m...,the goldfinch,39.676220
684,28862387,275,The Bear and the Nightingale,2067,https://www.goodreads.com/book/show/28862387-t...,https://images.gr-assets.com/books/1469305189m...,the bear and the nightingale,36.586841
9046,26856502,35,"Vengeful (Villains, #2)",35,https://www.goodreads.com/book/show/26856502-v...,https://s.gr-assets.com/assets/nophoto/book/11...,vengeful villains 2,35.000000
2566,34050917,100,The Girl in the Tower (The Winternight Trilogy...,331,https://www.goodreads.com/book/show/34050917-t...,https://images.gr-assets.com/books/1501159995m...,the girl in the tower the winternight trilogy 2,30.211480
6814,24909347,45,"Obsidio (The Illuminae Files, #3)",82,https://www.goodreads.com/book/show/24909347-o...,https://images.gr-assets.com/books/1501704611m...,obsidio the illuminae files 3,24.695122
10241,30694168,31,"Shadowsong (Wintersong, #2)",43,https://www.goodreads.com/book/show/30694168-s...,https://images.gr-assets.com/books/1496960059m...,shadowsong wintersong 2,22.348837


In [53]:
popular_recs = all_recs[all_recs["book_count"] > 75].sort_values("score", ascending=False)

In [55]:
def clickable(val):
    return '<a target="_blank" href={}"> Goodreads </a>'.format(val)

def cover(val):
    return '<img src="{}" width=60></image>'.format(val)

popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'cover':cover, 'url': clickable})


,book_id,book_count,title,ratings,url,cover,modified_title,score
2566,34050917,100,The Girl in the Tower (The Winternight Trilogy #2),331,Goodreads,,the girl in the tower the winternight trilogy 2,30.211480
3140,26032825,85,"The Cruel Prince (The Folk of the Air, #1)",400,Goodreads,,the cruel prince the folk of the air 1,18.062500
1151,21032488,190,"Doors of Stone (The Kingkiller Chronicle, #3)",2059,Goodreads,,doors of stone the kingkiller chronicle 3,17.532783
862,12111823,235,"The Winds of Winter (A Song of Ice and Fire, #6)",4404,Goodreads,,the winds of winter a song of ice and fire 6,12.539737
1893,34928122,129,Artemis,1478,Goodreads,,artemis,11.259134
2595,6382055,100,"A Dream of Spring (A Song of Ice and Fire, #7)",914,Goodreads,,a dream of spring a song of ice and fire 7,10.940919
2435,33590260,105,"Untitled (Throne of Glass, #7)",1190,Goodreads,,untitled throne of glass 7,9.264706
1055,32920226,201,"Sing, Unburied, Sing",4592,Goodreads,,sing unburied sing,8.798127
2796,28421168,94,"Renegades (Renegades, #1)",1069,Goodreads,,renegades renegades 1,8.265669
2705,26114477,97,The Geek Feminist Revolution,1150,Goodreads,,the geek feminist revolution,8.181739
